<a href="https://colab.research.google.com/github/porori1224/2024_Chosun_osp_Deepfake-detection-Challenge/blob/porori1224/KMS_Try_Code/KMS_Try5_DFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
# check the current path
print(os.getcwd()) # /content

# You should copy the path : 왼쪽 폴더에서 오른쪽 버튼 후 경로복사
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW')

print(os.getcwd()) # path has been changed

/content/gdrive/.shortcut-targets-by-id/10WgLiaOCZzMlANImGj_dPtI-hdfRCUe_/2024_OSP_SW
/content/gdrive/.shortcut-targets-by-id/10WgLiaOCZzMlANImGj_dPtI-hdfRCUe_/2024_OSP_SW


In [ ]:
!pip install torch torchvision torchaudio
!pip install retinaface-pytorch
!pip install efficientnet-pytorch
!pip install albumentations
!pip install timm

In [ ]:
import json
import pandas as pd

json_file = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos/metadata.json'
df = pd.read_json(json_file)
df = df.T

df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [ ]:
df.loc['aagfhgtpmv.mp4'].label

'FAKE'

In [ ]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from retinaface.pre_trained_models import get_model as get_retinaface_model
import albumentations as A
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import json

# RetinaFace 모델 로드
retinaface_model = get_retinaface_model("resnet50_2020-07-20", max_size=224)
retinaface_model.eval()

# 사용자 데이터셋 경로
train_dir = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos'
test_dir = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/test_videos'

# JSON 파일을 DataFrame으로 불러와 라벨 정보를 설정
json_file = '/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/train_sample_videos/metadata.json'
df = pd.read_json(json_file).T  # Transpose to set filenames as index


class RetinaFacePreprocess:
    def __call__(self, image):
        if image is None or image.size == 0:  # image가 비어 있거나 None인 경우 체크
            print("Warning: No face detected and input image is empty. Returning blank image.")
            return np.zeros((244, 244, 3), dtype=np.uint8)  # 빈 이미지 반환 (244x244 크기, RGB 채널)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faces = retinaface_model.predict_jsons(image)  # 얼굴 검출
        if faces:
            try:
                x1, y1, x2, y2 = faces[0]["bbox"]
                image = image[int(y1):int(y2), int(x1):int(x2)]
            except ValueError:
                pass

        if image.size == 0:  # 얼굴이 검출되지 않아 image가 비어 있는 경우
            print("Warning: No face detected. Using a blank image.")
            return np.zeros((244, 244, 3), dtype=np.uint8)  # 빈 이미지 반환 (244x244 크기, RGB 채널)

        return cv2.resize(image, (244, 244))  # (244, 244) 크기로 리사이즈


# Albumentations 전용 변환 설정
albumentations_transform_train = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.CoarseDropout(num_holes_range=(1, 1),
                    hole_height_range=(10, 20),
                    hole_width_range=(10, 20),
                    fill_value=0,
                    p=0.8),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])
albumentations_transform_val = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

class VideoFrameDataset(Dataset):
    def __init__(self, video_dir, df, preprocess, albumentations_transform=None, num_frames=10, display=False, use_dummy_label=False):
        self.video_dir = video_dir
        self.df = df  # DataFrame을 통해 라벨 설정
        self.preprocess = preprocess
        self.albumentations_transform = albumentations_transform
        self.to_tensor = ToTensor()
        self.num_frames = num_frames
        self.video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
        self.display = display
        self.use_dummy_label = use_dummy_label
        self.frames_and_labels = self._load_frames_and_labels()  # 1500개의 (프레임, 라벨) 리스트 생성

    def _load_frames_and_labels(self):
        frames_and_labels = []
        for video_file in self.video_files:
            video_path = os.path.join(self.video_dir, video_file)
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_interval = max(total_frames // self.num_frames, 1)

            for i in range(self.num_frames):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
                success, frame = cap.read()
                if not success:
                    print(f"Warning: Could not read frame {i} in {video_path}. Skipping.")
                    continue

                # 얼굴 추출 및 전처리
                face = self.preprocess(frame)
                if self.albumentations_transform:
                    face = self.albumentations_transform(image=face)['image']
                face_tensor = self.to_tensor(face)

                # 라벨 설정
                if not self.use_dummy_label:
                    label_name = self.df.loc[video_file, 'label']
                    label = 1 if label_name == 'FAKE' else 0
                else:
                    label = 0  # 더미 라벨 설정

                frames_and_labels.append((face_tensor, label))

                if len(frames_and_labels) >= 1500:  # 총 1500개 항목만 필요
                    break
            cap.release()
            if len(frames_and_labels) >= 1500:
                break
        return frames_and_labels

    def __len__(self):
        return len(self.frames_and_labels)  # 총 1500개의 샘플을 반환

    def __getitem__(self, idx):
        return self.frames_and_labels[idx]  # [3, 244, 244] 크기의 텐서와 float 라벨 반환

# train_dataset과 test_dataset 정의
train_dataset = VideoFrameDataset(
    train_dir, df, preprocess=RetinaFacePreprocess(), albumentations_transform=albumentations_transform_train, display=False
)
test_dataset = VideoFrameDataset(
    test_dir, df, preprocess=RetinaFacePreprocess(), albumentations_transform=albumentations_transform_val, display=True, use_dummy_label=True
)

# train_dataset 개별 데이터 확인
for frames, labels in train_dataset:
    print(f"Single Frame shape: {frames.shape}")  # 예상: [3, 244, 244]
    print(f"Single Label shape: {labels}")  # 예상: 단일 float 값
    break

# DataLoader로 배치 단위 데이터 확인
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 배치 단위 데이터 확인
for frames, labels in train_loader:
    print(f"Batch Frames shape: {frames.shape}")  # 예상: [64, 3, 244, 244]
    print(f"Batch Labels shape: {labels.shape}")  # 예상: [64]
    break

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/hub.py:870: FutureWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  return _legacy_zip_load(cached_file, model_dir, map_location, weights_only)


Single Frame shape: torch.Size([3, 244, 244])
Single Label shape: 1
Batch Frames shape: torch.Size([64, 3, 244, 244])
Batch Labels shape: torch.Size([64])


In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_pre_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

In [ ]:
import torch
from torchsummary import summary
from efficientnet_pytorch import EfficientNet

import torch.nn as nn

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=1)

## freeze feature extractor ,Feature Extractor 고정 (Freeze):
## 학습 속도 및 효율을 높이기 위해 필요한 경우,
## 이 부분을 활성화하고 특정 레이어만 학습하도록 조정
#for param in model.parameters():
#    param.requires_grad = False
#del model._fc
#model._fc = nn.Linear(2560, 1)

summary(model, input_size=(3,224,224), device='cpu')


#model = torch.nn.parallel.DistributedDataParallel(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Loaded pretrained weights for efficientnet-b7
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13        

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        """
        Focal Loss 초기화
        Args:
            gamma (float): Focusing parameter. 기본값은 2.0.
            alpha (float): 클래스 가중치. 기본값은 0.25 (양성 클래스에 더 많은 가중치를 줄 때 유용).
            reduction (str): 손실 감소 방식 ('mean', 'sum', 'none'). 기본값은 'mean'.
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Focal Loss 계산
        Args:
            inputs (torch.Tensor): 모델의 예측값 (logits, Sigmoid를 사용하지 않음)
            targets (torch.Tensor): 실제 레이블 (0 또는 1)
        Returns:
            torch.Tensor: 계산된 Focal Loss 값
        """
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # 확률로 변환
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt

# Random Erasing 데이터 증강 정의
# p=0.5는 50% 확률로 적용, scale과 ratio는 지워질 부분의 크기와 비율 제어
#random_erasing = transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)

# 훈련 루프에서 데이터 증강을 적용하는 함수
def train_one_epoch(model, dataloader, criterion, optimizer, device, log_interval=100):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0  # 누적 손실 초기화
    correct = 0  # 맞은 예측 수 초기화
    total = 0  # 총 예측 수 초기화

    # 데이터로더에서 배치별로 데이터를 가져옴
    for i, (inputs, labels) in enumerate(dataloader):
        # 입력 이미지에 랜덤 지우기 증강 적용
        inputs = inputs.to(device)  # 입력 데이터를 장치(GPU 또는 CPU)로 이동
#        for j in range(inputs.size(0)):
#            inputs[j] = random_erasing(inputs[j])  # 각 이미지를 랜덤 지우기 증강 적용

        # 첫 번째 배치의 몇 개의 샘플을 시각화하여 증강 확인
#        if i == 0:  # 첫 번째 배치에만 샘플을 출력
#            print("Random Erasing이 적용된 샘플들을 확인합니다:")
#            for k in range(min(5, inputs.size(0))):  # 최대 5개의 샘플을 출력
#                plt.imshow(inputs[k].cpu().permute(1, 2, 0).numpy())  # 채널 순서를 변경하여 이미지 출력
#                plt.show()  # 이미지를 화면에 표시

        # 라벨 데이터를 장치로 이동하고 형식 변환
        labels = labels.to(device).unsqueeze(1).float()  # 라벨을 (batch_size, 2) 형식에 맞춰 변환

        optimizer.zero_grad()  # 옵티마이저의 경사도 초기화
        outputs = model(inputs)  # 모델을 통해 예측 수행
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파를 통해 경사도 계산
        optimizer.step()  # 옵티마이저로 가중치 업데이트

        # 예측 결과를 이진 분류에 맞게 0과 1로 변환
        outputs[outputs >= 0.5] = 1
        outputs[outputs < 0.5] = 0

        # 누적 손실 계산
        running_loss += loss.item() * inputs.size(0)
        # 맞춘 예측 수를 합산
        correct += outputs.eq(labels).int().sum()

    # 평균 손실과 정확도를 반환
    return running_loss / len(dataloader), 100 * correct / len(dataloader.dataset)

# 평가 함수는 Random Erasing이 적용되지 않으므로 기존 코드와 동일하게 유지
def evaluate(model, dataloader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정
    running_loss = 0.0  # 누적 손실 초기화
    correct = 0  # 맞은 예측 수 초기화
    total = 0  # 총 예측 수 초기화

    with torch.no_grad():  # 평가에서는 경사도를 계산하지 않음
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels = labels.unsqueeze(1).float()
            loss = criterion(outputs, labels)

            # 예측 결과를 이진 분류에 맞게 0과 1로 변환
            outputs[outputs >= 0.5] = 1
            outputs[outputs < 0.5] = 0

            # 누적 손실 계산
            running_loss += loss.item() * inputs.size(0)
            # 맞춘 예측 수를 합산
            correct += outputs.eq(labels).int().sum()

    # 평균 손실과 정확도를 반환
    return running_loss / len(dataloader), 100 * correct / len(dataloader.dataset)

# 모델: EfficientNet-B7
# 손실함수: Focal Loss
#데이터증강: Random Augmentation

In [ ]:
import torch
from torch.utils.data import DataLoader
import copy
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from efficientnet_pytorch import EfficientNet

# EfficientNet 모델 정의 및 로드
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=1)

# 모델 요약 정보 출력
summary(model, input_size=(3, 224, 224), device='cpu')

# GPU 사용 여부 확인 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Focal Loss와 옵티마이저 설정
criterion = FocalLoss(gamma=2.0, alpha=0.25)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 손실 함수와 옵티마이저 확인
#print("Loss Function:", criterion)
#print("Optimizer:", optimizer)

# Train과 Test DataLoader 설정
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Train 함수 정의
def train_one_epoch(model, dataloader, criterion, optimizer, device, log_interval=100):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()  # 이진 분류이므로 labels의 형태를 (batch_size, 1)로 맞춤

        optimizer.zero_grad()

        # 모델에 전체 입력 전달
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # 로그 간격마다 출력
        if (batch_idx + 1) % log_interval == 0:
            print(f'Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Test 함수 정의
def evaluate(model, dataloader, device):
    model.eval()  # 모델을 평가 모드로 전환
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            # `inputs`만 추출하여 `.to(device)` 사용
            inputs = batch[0].to(device)

            outputs = model(inputs)
            preds = (outputs >= 0).float().cpu().numpy()
            predictions.extend(preds)

    return predictions

# 학습 루프 예제 (기본적인 구조)
#num_epochs = 20  # 에포크 수 설정
#for epoch in range(num_epochs):
#    model.train()  # 모델을 학습 모드로 설정
#    running_loss = 0.0

    # 예시 데이터 로드 (사용자 데이터셋으로 대체 필요)
#    for inputs, labels in train_dataset:
#        inputs, labels = inputs.to(device), labels.to(device)  # 데이터도 GPU로 전송

#        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화

        # 순전파
 #       outputs = model(inputs)
 #       loss = criterion(outputs, labels.float())  # Focal Loss 계산

        # 역전파 및 최적화
 #       loss.backward()
 #       optimizer.step()

#        running_loss += loss.item()

    # 에포크당 평균 손실 출력
#    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

#print("Training complete")

# 학습 설정
num_epochs = 20
best_val_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    # Train 데이터셋 학습 (라벨이 있음)
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # Test 데이터셋 예측 (라벨이 없음)
    predictions = evaluate(model, test_loader, device)

    # 예측값 출력 (처음 10개만 예시로 표시)
    print(f"Test Predictions (first 10): {predictions[:10]}")

    # 모델의 성능 개선 여부 확인
    if train_acc > best_val_acc:
        best_val_acc = train_acc
        best_model_wts = copy.deepcopy(model.state_dict())

# 가장 성능 좋은 가중치로 모델 업데이트
model.load_state_dict(best_model_wts)

Loaded pretrained weights for efficientnet-b7
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13        

<All keys matched successfully>

# **EfficientNet+Xception 모델 사용**

In [ ]:
!pip install pretrainedmodels

In [ ]:
import pretrainedmodels
import ssl

# SSL 인증서 검증 건너뛰기
ssl._create_default_https_context = ssl._create_unverified_context

# Xception 모델 로드
xception = pretrainedmodels.__dict__['xception'](pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth
100%|██████████| 87.4M/87.4M [04:17<00:00, 356kB/s]


In [ ]:
import pretrainedmodels
import torch.nn as nn

# Xception 모델 불러오기
xception = pretrainedmodels.__dict__['xception'](pretrained='imagenet')

# 이진 분류를 위해 출력 레이어를 1개로 수정
xception.last_linear = nn.Linear(xception.last_linear.in_features, 1)

In [ ]:
import torch
from torch.utils.data import DataLoader
import copy
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from efficientnet_pytorch import EfficientNet
import pretrainedmodels  # Xception 모델을 위해 필요

# EfficientNet 모델 정의 및 로드
efficientnet = EfficientNet.from_pretrained('efficientnet-b7', num_classes=1)
summary(efficientnet, input_size=(3, 224, 224), device='cpu')

# Xception 모델 정의 및 로드
xception = pretrainedmodels.__dict__['xception'](pretrained='imagenet')
xception.last_linear = nn.Linear(xception.last_linear.in_features, 1)  # 이진 분류를 위해 출력 수정
summary(xception, input_size=(3, 299, 299), device='cpu')  # Xception의 기본 입력 크기는 299x299

# GPU 사용 여부 확인 및 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet = efficientnet.to(device)
xception = xception.to(device)

# Focal Loss와 옵티마이저 설정
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

criterion = FocalLoss(gamma=2.0, alpha=0.25)
optimizer = optim.Adam(list(efficientnet.parameters()) + list(xception.parameters()), lr=1e-4)

# Train과 Test DataLoader 설정
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Train 함수 정의
def train_one_epoch(efficientnet, xception, dataloader, criterion, optimizer, device, log_interval=100):
    efficientnet.train()
    xception.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()

        optimizer.zero_grad()

        # EfficientNet과 Xception의 각각의 예측값을 계산
        eff_out = efficientnet(inputs)
        xcep_out = xception(nn.functional.interpolate(inputs, size=(299, 299)))  # Xception은 299x299 입력 필요

        # 두 모델의 출력을 평균하여 최종 예측 생성
        outputs = (eff_out + xcep_out) / 2
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # 로그 간격마다 출력
        if (batch_idx + 1) % log_interval == 0:
            print(f'Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Test 함수 정의
def evaluate(efficientnet, xception, dataloader, device):
    efficientnet.eval()
    xception.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)

            eff_out = efficientnet(inputs)
            xcep_out = xception(nn.functional.interpolate(inputs, size=(299, 299)))
            outputs = (eff_out + xcep_out) / 2  # 두 모델의 예측을 평균
            preds = (outputs >= 0).float().cpu().numpy()
            predictions.extend(preds)

    return predictions

# 학습 설정
num_epochs = 20
best_val_acc = 0.0
best_model_wts = {
    "efficientnet": copy.deepcopy(efficientnet.state_dict()),
    "xception": copy.deepcopy(xception.state_dict())
}

for epoch in range(num_epochs):
    # Train 데이터셋 학습
    train_loss, train_acc = train_one_epoch(efficientnet, xception, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # Test 데이터셋 예측
    predictions = evaluate(efficientnet, xception, test_loader, device)

    # 예측값 출력 (처음 10개만 예시로 표시)
    print(f"Test Predictions (first 10): {predictions[:10]}")

    # 모델의 성능 개선 여부 확인
    if train_acc > best_val_acc:
        best_val_acc = train_acc
        best_model_wts["efficientnet"] = copy.deepcopy(efficientnet.state_dict())
        best_model_wts["xception"] = copy.deepcopy(xception.state_dict())

# 가장 성능 좋은 가중치로 모델 업데이트
efficientnet.load_state_dict(best_model_wts["efficientnet"])
xception.load_state_dict(best_model_wts["xception"])

Loaded pretrained weights for efficientnet-b7
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13        

<All keys matched successfully>

**모델: ensemble_model에 Focal Loss를 적용
-> 불균형 데이터에 대한 성능을 강화**

**데이터 증강: albumentations로 Random Rotation, Horizontal Flip, Coarse Dropout, Random Brightness Contrast 등 다양한 증강을 적용하여 일반화 성능을 강화**

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary
from efficientnet_pytorch import EfficientNet
import timm  # timm 라이브러리로 Xception 모델 로드

# EfficientNet-B0 모델
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes=1):
        super(EfficientNetB0, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

# Xception 모델
class Xception(nn.Module):
    def __init__(self, num_classes=1):
        super(Xception, self).__init__()
        # timm을 사용하여 Xception 모델 로드
        self.model = timm.create_model('xception', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

# 앙상블 모델
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        # 두 모델의 예측값을 평균으로 앙상블
        output1 = self.model1(x)
        output2 = self.model2(x)
        return (output1 + output2) / 2  # 평균 예측값

# 모델 인스턴스 생성
effnet_model = EfficientNetB0(num_classes=1)
xception_model = Xception(num_classes=1)

# 앙상블 모델 생성
ensemble_model = EnsembleModel(effnet_model, xception_model)

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

# 모델 요약 출력
summary(ensemble_model, input_size=(3, 224, 224), device='cuda' if torch.cuda.is_available() else 'cpu')

# 학습용 DataLoader 예시
from torch.utils.data import DataLoader

batch_size = 32  # 배치 크기 설정 (GPU 메모리와 학습 속도에 따라 조정)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# 배치 데이터 확인
for frames, labels in train_loader:
    print(f"배치 프레임의 크기: {frames.shape}")  # 예상: [32, 3, 224, 224]
    print(f"배치 레이블의 크기: {labels.shape}")  # 예상: [32]
    break


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 35.8MB/s]
/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [ ]:
import copy
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)  # 이미 정의된 앙상블 모델

# Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# 손실 함수 및 옵티마이저 설정
criterion = FocalLoss(gamma=2.0, alpha=0.25)  # Focal Loss 사용
optimizer = optim.Adam(ensemble_model.parameters(), lr=1e-4)

# 데이터 증강 설정
albumentations_transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.CoarseDropout(max_holes=1, max_height=20, max_width=20, p=0.8),  # 일부 영역 마스킹
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Train과 Test DataLoader 설정
batch_size = 32  # 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Train 함수 정의
def train_one_epoch(model, dataloader, criterion, optimizer, device, log_interval=100):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()  # 이진 분류에 맞게 라벨을 (batch_size, 1)로 변환

        optimizer.zero_grad()

        # 모델에 입력 전달
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # 로그 간격마다 출력
        if (batch_idx + 1) % log_interval == 0:
            print(f'Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Test 함수 정의
def evaluate(model, dataloader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            outputs = model(inputs)
            preds = (outputs >= 0).float().cpu().numpy()
            predictions.extend(preds)

    return predictions

# 학습 설정
num_epochs = 20
best_val_acc = 0.0
best_model_wts = copy.deepcopy(ensemble_model.state_dict())

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(ensemble_model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # Test 데이터셋 예측
    predictions = evaluate(ensemble_model, test_loader, device)
    print(f"Test Predictions (first 10): {predictions[:10]}")

    # 성능 개선 확인
    if train_acc > best_val_acc:
        best_val_acc = train_acc
        best_model_wts = copy.deepcopy(ensemble_model.state_dict())

# 가장 성능 좋은 가중치로 모델 업데이트
ensemble_model.load_state_dict(best_model_wts)

Epoch 1/20
Train Loss: 0.0244, Train Acc: 84.8667
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]
Epoch 2/20
Train Loss: 0.0072, Train Acc: 97.0667
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]
Epoch 3/20
Train Loss: 0.0039, Train Acc: 98.0000
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), arra

<All keys matched successfully>

# **Xception + Focal Loss + Random Augmentation + 일부 영역 마스킹 (Coarse Dropout)**

In [ ]:
import copy
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 모델 정의 및 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet = EfficientNet.from_pretrained('efficientnet-b7', num_classes=1).to(device)

# 손실 함수 정의
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean() if self.reduction == 'mean' else F_loss.sum()

criterion = FocalLoss(gamma=2.0, alpha=0.25)
optimizer = optim.Adam(efficientnet.parameters(), lr=1e-4)

# 데이터 증강 설정
albumentations_transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Train 함수 정의
def train_one_epoch(model, dataloader, criterion, optimizer, device, log_interval=100):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        if (batch_idx + 1) % log_interval == 0:
            print(f'Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Test 함수 정의
def evaluate(model, dataloader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1).float()
            outputs = model(inputs)
            preds = (outputs >= 0).float().cpu().numpy()
            predictions.extend(preds)

    return predictions

# 학습 루프
num_epochs = 20
best_val_acc = 0.0
best_model_wts = copy.deepcopy(efficientnet.state_dict())

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(efficientnet, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    predictions = evaluate(efficientnet, test_loader, device)
    print(f"Test Predictions (first 10): {predictions[:10]}")

    if train_acc > best_val_acc:
        best_val_acc = train_acc
        best_model_wts = copy.deepcopy(efficientnet.state_dict())

# 가장 성능 좋은 가중치로 모델 업데이트
efficientnet.load_state_dict(best_model_wts)

Loaded pretrained weights for efficientnet-b7
Epoch 1/20
Train Loss: 0.0318, Train Acc: 77.0000
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]
Epoch 2/20
Train Loss: 0.0106, Train Acc: 96.4000
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]
Epoch 3/20
Train Loss: 0.0057, Train Acc: 97.2667
Test Predictions (first 10): [array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dt

<All keys matched successfully>